### Initialization including packages and data

#### Packages

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

#### Reading data into dataframes

In [47]:
df_genTech = pd.read_csv(r"data\TechnicalDataofGeneratingUnits-Table1.csv")

df_gen = pd.read_csv(r"data\CostsandInitialStateofGeneratingUnits-Table2.csv")

df_load = pd.read_csv(r"data\LoadProfile-Table3.csv")

df_demand = pd.read_csv(r"data\NodeLocationandDistributionoftheTotalSystemDemand-Table4.csv")
df_demand["Percent"] = df_demand["Percent"]/100

df_renew = pd.read_csv(r"data\scen_zone1.out", nrows = 24)
df_renew = df_renew.iloc[:, 1:]

df_bid = pd.read_csv(r"data\demand_utility_prices_24h.csv",index_col="Hour")


### Copper-Plate, Single Hour

#### System model

In [ ]:
# Initialize Gurobi model
model1 = gp.Model('Copperplate')

#Hours in a day:
time = 12

#Constants
cap_re = 200 #Capacity on the renewable generators
C_ren = 0 #Cost of renewable generation

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

# Variables
Pgen = model1.addVars(N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model1.addVars(N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model1.addVars(N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")


# Demand Capacity Constraints
model1.addConstrs((Pd[d] <= df_demand.loc[d,"Percent"] * df_load.loc[time,"Demand"]
                for d in range(N_d)), name ="Demand Max Capacity")

model1.addConstrs((Pd[d] >= 0
                for d in range(N_d)), name ="Demand Min Capacity")

# Generator Capacity Constraints
model1.addConstrs((Pgen[gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)), name ="Generator Max Capacity")

model1.addConstrs((Pgen[gen] >= 0
                for gen in range(N_gen)), name ="Generator Min Capacity")
#model.addConstrs((Pgen[gen] >= df_genTech.loc[gen,"Pmin"]
#                for gen in range(N_gen)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model1.addConstrs((Pw[ren] <= df_renew.iloc[time,ren] * cap_re
                for ren in range(N_ren)), name ="Renewable Max Capacity")

model1.addConstrs((Pw[ren] >= 0
                for ren in range(N_ren)), name ="Renewable Min Capacity")


# Power balance constraint
PB_single = model1.addConstr(sum(Pgen[gen] for gen in range(N_gen))
              + sum(Pw[ren] for ren in range(N_ren)) 
              - sum(Pd[d] for d in range(N_d)) == 0, name="Power Balance")


model1.setObjective(sum(Pd[d] * df_bid.iloc[time,d] for d in range(N_d))
    - sum(Pgen[gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
    - sum(Pw[ren] * C_ren for ren in range(N_ren)),
    sense=GRB.MAXIMIZE
)
model1.update()

# Solve the optimization problem
model1.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 71 rows, 35 columns and 105 nonzeros
Model fingerprint: 0xae7e0f62
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 70 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 26 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7679965e+04   1.877948e+02   0.000000e+00      0s
       1    8.5664681e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.566468131e+04


#### Data processing and analysis

In [49]:
clear_price_single = []
# Clearing price for each hour:
clear_price_single.append(PB_single.Pi)

### Copper-Plate, Multiple Hour (including storage unit)

#### System model

##### Copper-Plate, Multiple Hour (excluding storage unit)

In [ ]:
# Initialize Gurobi model wo storage
model22 = gp.Model('Copperplate-multi-nostorage')

#Hours in a day:
T = len(df_load)

#Constants
cap_re = 200 #Capacity on the renewable generators

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

# Variables
Pgen = model22.addVars(T,N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model22.addVars(T,N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model22.addVars(T,N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")

# Demand Capacity Constraints
model22.addConstrs((Pd[t,d] <= df_demand.loc[d,"Percent"] * df_load.loc[t,"Demand"]
                for d in range(N_d)
                for t in range(T)), name ="Demand Max Capacity")

model22.addConstrs((Pd[t,d] >= 0
                for d in range(N_d)
                for t in range(T)), name ="Demand Min Capacity")


# Generator Capacity Constraints
model22.addConstrs((Pgen[t,gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Max Capacity")

#model.addConstrs((Pgen[t,gen] >= df_genTech.loc[gen,"Pmin"] DO WE NEED MINIMUM DISPATCH OF GENERATORS?
#                for gen in range(N_gen)
#                for t in range(T)), name ="Generator Min Capacity")
model22.addConstrs((Pgen[t,gen] >= 0
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model22.addConstrs((Pw[t,ren] <= df_renew.iloc[t,ren] * cap_re
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Max Capacity")

model22.addConstrs((Pw[t,ren] >= 0
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Min Capacity")


# Power balance constraint
# Create power balance constraint for each t in T
# so that we dont allow overproduction in one hour to be used in another hour
power_balance22 = model22.addConstrs((
                sum(Pgen[t,gen] for gen in range(N_gen))
              + sum(Pw[t,ren] for ren in range(N_ren))
              == 
              sum(Pd[t,d] for d in range(N_d)) for t in range(T)
             ), name="Power_Balance")

# Storage is not included in the objective function, described in the Note 2 of the assignment description
# It can be utilized, but does neither bid or offer on energy market

# Objective function
model22.setObjective(sum(
                 sum(Pd[t,d] * df_bid.iloc[t,d] for d in range(N_d))
                 - sum(Pgen[t,gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
                 - sum(Pw[t,ren] * 0 for ren in range(N_ren)) for t in range(T)), sense=GRB.MAXIMIZE)

model22.update()

# Solve the optimization problem
model22.optimize()




Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1704 rows, 840 columns and 2520 nonzeros
Model fingerprint: 0xf1beae7b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 1680 rows and 216 columns
Presolve time: 0.02s
Presolved: 24 rows, 624 columns, 624 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1137983e+06   3.301092e+03   0.000000e+00      0s
      24    1.9192215e+06   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.919221478e+06


In [51]:
# Init
clear_price22 = []
total_operating_price22 = 0
total_profit_prod22 = []
profit_conv22 = []
profit_wind22 = []
consumer_surplus22 = []
social_welfare22 = 0
# Clearing price for each hour:
for t in range(T):
    clear_price22.append(power_balance22[t].Pi)


# Total Operating price:
P_gen_list22 = [[Pgen[t, gen].X for gen in range(N_gen)] for t in range(T)]
P_wind_list22 = [[Pw[t, ren].X for ren in range(N_ren)] for t in range(T)]
P_D_list22 = [[Pd[t, d].X for d in range(N_d)] for t in range(T)]

for t in range(T):
    for gen in range(N_gen):
        total_operating_price22 += -1 * (clear_price22[t] * P_gen_list22[t][gen])

# Total profit for each producer:
profit_conv22 = [[-1 * (clear_price22[t] - df_gen.loc[gen, "Ci"]) * P_gen_list22[t][gen] 
            for gen in range(N_gen)]
            for t in range(T)]    

profit_wind22 = [[-1 * (clear_price22[t] - 0) * P_wind_list22[t][ren] 
            for ren in range(N_ren)]
            for t in range(T)]    

consumer_surplus22 = [[(df_bid.iloc[t,d] - clear_price22[t]) * P_D_list22[t][d] 
            for d in range(N_d)]
            for t in range(T)] 

# Social welfare
# Sum of profits for all generators + sum of clearing price and bid price for demand
social_welfare22 = sum(sum(profit_conv22,[])) + sum(sum(profit_wind22,[])) + sum(sum(consumer_surplus22,[]))



##### Copper-Plate, Multiple Hour (including storage unit)

In [ ]:
# Initialize Gurobi model
model21 = gp.Model('Copperplate-multi-storage')


#Hours in a day:
T = len(df_load)

#Constants
cap_re = 200 #Capacity on the renewable generators

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

#Constants for storage unit
E = 100 #MWh
P_ch = 50 #MW
P_dis = 50 #MW
eta_ch = 0.9 
eta_dis = 0.95

# Variables
Pgen = model21.addVars(T,N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model21.addVars(T,N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model21.addVars(T,N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")

Pch = model21.addVars(T, vtype=gp.GRB.CONTINUOUS, name="P_charge")
Pdis = model21.addVars(T, vtype=gp.GRB.CONTINUOUS, name="P_discharge")
e = model21.addVars(T, vtype=gp.GRB.CONTINUOUS, name="Storage")


# Demand Capacity Constraints
model21.addConstrs((Pd[t,d] <= df_demand.loc[d,"Percent"] * df_load.loc[t,"Demand"]
                for d in range(N_d)
                for t in range(T)), name ="Demand Max Capacity")

model21.addConstrs((Pd[t,d] >= 0
                for d in range(N_d)
                for t in range(T)), name ="Demand Min Capacity")


# Generator Capacity Constraints
model21.addConstrs((Pgen[t,gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Max Capacity")

#model.addConstrs((Pgen[t,gen] >= df_genTech.loc[gen,"Pmin"] DO WE NEED MINIMUM DISPATCH OF GENERATORS?
#                for gen in range(N_gen)
#                for t in range(T)), name ="Generator Min Capacity")
model21.addConstrs((Pgen[t,gen] >= 0
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model21.addConstrs((Pw[t,ren] <= df_renew.iloc[t,ren] * cap_re
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Max Capacity")

model21.addConstrs((Pw[t,ren] >= 0
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Min Capacity")


# Power balance constraint
# Create power balance constraint for each t in T
# so that we dont allow overproduction in one hour to be used in another hour
power_balance21 = model21.addConstrs((
                sum(Pgen[t,gen] for gen in range(N_gen))
              + sum(Pw[t,ren] for ren in range(N_ren))
              + Pdis[t]
              == 
                Pch[t]
              + sum(Pd[t,d] for d in range(N_d)) for t in range(T)
             ), name="Power_Balance")

#Adding the storage unit

# Storage charging constraints
model21.addConstrs((Pch[t] >= 0
                for t in range(T)), name ="Charging Min Capacity")

model21.addConstrs((Pch[t] <= P_ch
                for t in range(T)), name ="Charging Max Capacity")

# Storage discharging constraints
model21.addConstrs((Pdis[t] >= 0
                for t in range(T)), name ="Discharging Min Capacity")

model21.addConstrs((Pdis[t] <= P_dis
                for t in range(T)), name ="Discharging Max Capacity")

# Storage capacity constraints
model21.addConstrs((e[t] >= 0
                  for t in range(T)), name ="Storage Min Capacity")

model21.addConstrs((e[t] <= E
                  for t in range(T)), name ="Storage Max Capacity")

# Modelling charge and discharge
model21.addConstrs((e[t] == e[t-1] + eta_ch * Pch[t] - (1/eta_dis) * Pdis[t] for t in range(1,T)), name="Storage Balance")

# Initial condition of the storage unit
model21.addConstr(e[0] == 0.5 * E, name="Initial Capacity")


# Storage is not included in the objective function, described in the Note 2 of the assignment description
# It can be utilized, but does neither bid or offer on energy market

# Objective function
model21.setObjective(sum(
                 sum(Pd[t,d] * df_bid.iloc[t,d] for d in range(N_d))
                 - sum(Pgen[t,gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
                 - sum(Pw[t,ren] * 0 for ren in range(N_ren)) for t in range(T)), sense=GRB.MAXIMIZE)

model21.update()

# Solve the optimization problem
model21.optimize()




Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1872 rows, 912 columns and 2805 nonzeros
Model fingerprint: 0x5c982419
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [5e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 1825 rows and 786 columns
Presolve time: 0.02s
Presolved: 47 rows, 126 columns, 194 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9595120e+06   1.029357e+03   0.000000e+00      0s
      59    1.9204781e+06   0.000000e+00   0.000000e+00      0s

Solved in 59 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.920478139e+06


#### Data processing and analysis

In [53]:
# Init
clear_price21 = []
clear_price22 = []
total_operating_price21 = 0
total_profit_prod21 = []
profit_conv21 = []
profit_wind21 = []
consumer_surplus21 = []
social_welfare21 = 0
# Clearing price for each hour:
for t in range(T):
    clear_price21.append(power_balance21[t].Pi)


# Total Operating price:
P_gen_list21 = [[Pgen[t, gen].X for gen in range(N_gen)] for t in range(T)]
P_wind_list21 = [[Pw[t, ren].X for ren in range(N_ren)] for t in range(T)]
P_D_list21 = [[Pd[t, d].X for d in range(N_d)] for t in range(T)]

for t in range(T):
    for gen in range(N_gen):
        total_operating_price21 += -1 * (clear_price21[t] * P_gen_list21[t][gen])

# Total profit for each producer:
profit_conv21 = [[-1 * (clear_price21[t] - df_gen.loc[gen, "Ci"]) * P_gen_list21[t][gen] 
            for gen in range(N_gen)]
            for t in range(T)]    

profit_wind21 = [[-1 * (clear_price21[t] - 0) * P_wind_list21[t][ren] 
            for ren in range(N_ren)]
            for t in range(T)]    

consumer_surplus21 = [[(df_bid.iloc[t,d] - clear_price21[t]) * P_D_list21[t][d] 
            for d in range(N_d)]
            for t in range(T)] 

# Social welfare
# Sum of profits for all generators + sum of clearing price and bid price for demand
social_welfare21 = sum(sum(profit_conv21,[])) + sum(sum(profit_wind21,[])) + sum(sum(consumer_surplus21,[]))

# Profit of storage unit
# Difference between total operating price for model w and wo storage unit
# Clearing price for each hour:
storage_profit = -1 * (total_operating_price21 - total_operating_price22)
print(storage_profit)

1256.6611111112288
